# 02 – Silver Transformations

This notebook cleans and normalizes Bronze data into the **Silver layer**.


In [ ]:
from pyspark.sql.functions import col, trim, upper, to_timestamp

bronze_orders = spark.table("industrial_print.bronze.orders")
bronze_providers = spark.table("industrial_print.bronze.providers")
bronze_events = spark.table("industrial_print.bronze.events")

orders_clean = (
    bronze_orders
    .withColumn("order_date", to_timestamp("order_date"))
    .withColumn("status", upper(trim(col("status"))))
)

events_clean = (
    bronze_events
    .where(col("event_type").isNotNull())
    .withColumn("event_ts", to_timestamp("event_ts"))
)

orders_clean.write.format("delta").mode("overwrite").saveAsTable("industrial_print.silver.orders_clean")
bronze_providers.write.format("delta").mode("overwrite").saveAsTable("industrial_print.silver.providers_raw")
events_clean.write.format("delta").mode("overwrite").saveAsTable("industrial_print.silver.events_clean")